In [6]:
! python --version
! pip install -Uqq fastbook


Python 3.11.4


In [1]:
! pip show fastbook

Name: fastbook
Version: 0.0.29
Summary: Deep Learning for Coders, 2020
Home-page: https://github.com/fastai/fastbook
Author: Jeremy Howard and Sylvain Gugger
Author-email: info@fast.ai
License: Apache Software License 2.0
Location: /Users/sweiss/code/courses/fastbook/.venv/lib/python3.11/site-packages
Requires: datasets, fastai, graphviz, ipywidgets, packaging, pandas, pip, requests, sentencepiece, transformers
Required-by: 


In [2]:
import fastbook
fastbook.setup_book()

Matplotlib is building the font cache; this may take a moment.


In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [3]:
#hide
from fastbook import *
from fastai.vision.widgets import *

Our Azure credentials to power the bing search


In [4]:
import os
key = os.environ.get('AZURE_SEARCH_KEY', 'xxx')

We want to create a dog classifier.

In [ ]:
search_types = 'greyhound','labrador','pug','beagle','bulldog','poodle','rottweiler','german shepherd'

In [ ]:
path = Path('dogs')
if not path.exists():
    path.mkdir()
    for o in search_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} dog')
        download_images(dest, urls=results.attrgot('contentUrl'))

But, having searched for all of these dogs, we now need to make sure they have the images we'd expect.

In [ ]:
files = get_image_files(path)
failed = verify_images(files)
# Unlink the images that failed to open
failed.map(Path.unlink)

Now we turn these images into a dataset. First, we need to create the template, that is, the DataBlock.

In [ ]:
dogs = DataBlock(
  blocks=(ImageBlock, CategoryBlock),
  get_items=get_image_files,
  splitter=RandomSplitter(valid_pct=0.2, seed=42),
  get_y=parent_label,
  item_tfms=Resize(128))

Now, we feed the data into the DataBlock and create a DataLoader:

In [ ]:
dls = dogs.dataloaders(path)

Now, let's look at one of the batches.

In [ ]:
dls.valid.show_batch(max_n=8, nrows=2)

In [ ]:
dogs = dogs.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = dogs.dataloaders(path)
dls.valid.show_batch(max_n=8, nrows=2)

In [ ]:
dogs = dogs.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = dogs.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
dogs = dogs.new(item_tfms=RandomResizedCrop(224, min_scale=0.5))
dls = dogs.dataloaders(path)

Now, we learn/ fine-tune. 

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)
